In [1]:
from pycocotools.coco import COCO
import numpy as np

In [2]:
cocoGT = COCO("coco_annotations.json")
cocoDT = cocoGT.loadRes("output/points/predict.json")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!


In [3]:
imgids = cocoGT.getImgIds()
imgs = cocoGT.loadImgs(imgids)

In [4]:
def caculate_iou(pred, gt):
    """
    Calculate the Intersection over Union (IoU) between two masks.

    Args:
        pred (np.ndarray): Predicted mask.
        gt (np.ndarray): Ground truth mask.

    Returns:
        float: IoU value.
    """
    intersection = np.logical_and(pred, gt).sum()
    union = np.logical_or(pred, gt).sum() + 1e-7
    iou = intersection / union
    return iou

In [5]:
def caculate_dice(pred, gt):
    intersection = np.sum(pred * gt)
    dice_score = (2. * intersection) / (np.sum(pred) + np.sum(gt) + 1e-7)
    return dice_score

In [6]:
def caculate_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

In [7]:
video_id_set = set()

img_scores = []

for img in imgs:
    anns_dt = cocoDT.loadAnns(cocoDT.getAnnIds(imgIds=img["id"]))
    anns_gt = cocoGT.loadAnns(cocoGT.getAnnIds(imgIds=img["id"]))

    all_iou = {cat_id: [] for cat_id in cocoDT.getCatIds()}
    all_mae = {cat_id: [] for cat_id in cocoDT.getCatIds()}
    all_dice = {cat_id: [] for cat_id in cocoDT.getCatIds()}


    img_score = {
        "video_id": img["video_id"],
        "order_in_video": img["order_in_video"],
        "cat_scores": {cat_id: {"iou": None, "mae": None, "dice": None} for cat_id in cocoDT.getCatIds()},
        "avg_scores": {"iou": None, "mae": None, "dice": None},
    }

    for ann_dt in anns_dt:
        for ann_gt in anns_gt:

            if ann_dt["category_id"] == ann_gt["category_id"]:
                
                mask_dt = cocoDT.annToMask(ann_dt)
                mask_gt = cocoGT.annToMask(ann_gt)
                iou = caculate_iou(mask_dt, mask_gt)
                mae = caculate_mae(mask_dt, mask_gt)
                dice = caculate_dice(mask_dt, mask_gt)
                
                all_iou[ann_dt["category_id"]].append(iou)
                all_mae[ann_dt["category_id"]].append(mae)
                all_dice[ann_dt["category_id"]].append(dice)
                
    avg_iou = []
    avg_mae = []
    avg_dice = []
    
    for cat_id in cocoDT.getCatIds():
        img_score["cat_scores"][cat_id]["iou"] = np.nanmean(all_iou[cat_id]) if len(all_iou[cat_id]) > 0 else np.nan
        img_score["cat_scores"][cat_id]["mae"] = np.nanmean(all_mae[cat_id]) if len(all_mae[cat_id]) > 0 else np.nan
        img_score["cat_scores"][cat_id]["dice"] = np.nanmean(all_dice[cat_id]) if len(all_dice[cat_id]) > 0 else np.nan
        
        avg_iou.append(img_score["cat_scores"][cat_id]["iou"])
        avg_mae.append(img_score["cat_scores"][cat_id]["mae"])
        avg_dice.append(img_score["cat_scores"][cat_id]["dice"])

        
    img_score["avg_scores"]["iou"] = np.nanmean(avg_iou) if len(avg_iou) > 0 else np.nan
    img_score["avg_scores"]["mae"] = np.nanmean(avg_mae) if len(avg_mae) > 0 else np.nan
    img_score["avg_scores"]["dice"] = np.nanmean(avg_dice) if len(avg_dice) > 0 else np.nan
    
    video_id_set.add(img["video_id"])
    img_scores.append(img_score)



/tmp/ipykernel_338892/1372041296.py:50: RuntimeWarning: Mean of empty slice
  img_score["avg_scores"]["iou"] = np.nanmean(avg_iou) if len(avg_iou) > 0 else np.nan
/tmp/ipykernel_338892/1372041296.py:51: RuntimeWarning: Mean of empty slice
  img_score["avg_scores"]["mae"] = np.nanmean(avg_mae) if len(avg_mae) > 0 else np.nan
/tmp/ipykernel_338892/1372041296.py:52: RuntimeWarning: Mean of empty slice
  img_score["avg_scores"]["dice"] = np.nanmean(avg_dice) if len(avg_dice) > 0 else np.nan


In [8]:
def get_dicts_by_field_value(data, field_name, target_value):
    """
    从字典列表中获取特定字段具有指定值的所有字典。

    参数：
        data: 字典列表
        field_name: 要检查的字段名称
        target_value: 字段的目标值

    返回：
        包含符合条件的字典的新列表
    """
    return [item for item in data if item.get(field_name) == target_value]

In [9]:
video_scores = []

In [10]:
for video_id in video_id_set:
    frames = get_dicts_by_field_value(img_scores, "video_id", video_id)
    
    video_score = {
        "video_id": video_id,
        "frames": frames,
        "cat_scores": {cat_id: {"iou": None, "mae": None, "dice": None} for cat_id in cocoDT.getCatIds()},
        "avg_scores": {"iou": None, "mae": None, "dice": None},
    }
    
    all_iou = {cat_id: [] for cat_id in cocoDT.getCatIds()}
    all_mae = {cat_id: [] for cat_id in cocoDT.getCatIds()}
    all_dice = {cat_id: [] for cat_id in cocoDT.getCatIds()}
    
    for frame in frames:
        for category_id, category_data in frame["cat_scores"].items():
            all_iou[category_id].append(category_data["iou"]) 
            all_mae[category_id].append(category_data["mae"])
            all_dice[category_id].append(category_data["dice"])
            
    for key in cocoDT.getCatIds():
        video_score["cat_scores"][key]['iou'] = np.nanmean(all_iou[key]) 
        video_score["cat_scores"][key]['mae'] = np.nanmean(all_mae[key])
        video_score["cat_scores"][key]['dice'] = np.nanmean(all_dice[key])
        
    # print(all_iou)
        
    avg_iou = []
    avg_mae = []
    avg_dice = []
    
    for cat_id in cocoDT.getCatIds():
        video_score["cat_scores"][cat_id]["iou"] = np.nanmean(all_iou[cat_id]) if len(all_iou[cat_id]) > 0 else np.nan
        video_score["cat_scores"][cat_id]["mae"] = np.nanmean(all_mae[cat_id]) if len(all_mae[cat_id]) > 0 else np.nan
        video_score["cat_scores"][cat_id]["dice"] = np.nanmean(all_dice[cat_id]) if len(all_dice[cat_id]) > 0 else np.nan
        
        avg_iou.append(video_score["cat_scores"][cat_id]["iou"])
        avg_mae.append(video_score["cat_scores"][cat_id]["mae"])
        avg_dice.append(video_score["cat_scores"][cat_id]["dice"])

    
    video_score["avg_scores"]["iou"] = np.nanmean(avg_iou) if len(avg_iou) > 0 else np.nan
    video_score["avg_scores"]["mae"] = np.nanmean(avg_mae) if len(avg_mae) > 0 else np.nan
    video_score["avg_scores"]["dice"] = np.nanmean(avg_dice) if len(avg_dice) > 0 else np.nan
    
    video_scores.append(video_score)
    

/tmp/ipykernel_338892/747448669.py:22: RuntimeWarning: Mean of empty slice
  video_score["cat_scores"][key]['iou'] = np.nanmean(all_iou[key])
/tmp/ipykernel_338892/747448669.py:23: RuntimeWarning: Mean of empty slice
  video_score["cat_scores"][key]['mae'] = np.nanmean(all_mae[key])
/tmp/ipykernel_338892/747448669.py:24: RuntimeWarning: Mean of empty slice
  video_score["cat_scores"][key]['dice'] = np.nanmean(all_dice[key])
/tmp/ipykernel_338892/747448669.py:33: RuntimeWarning: Mean of empty slice
  video_score["cat_scores"][cat_id]["iou"] = np.nanmean(all_iou[cat_id]) if len(all_iou[cat_id]) > 0 else np.nan
/tmp/ipykernel_338892/747448669.py:34: RuntimeWarning: Mean of empty slice
  video_score["cat_scores"][cat_id]["mae"] = np.nanmean(all_mae[cat_id]) if len(all_mae[cat_id]) > 0 else np.nan
/tmp/ipykernel_338892/747448669.py:35: RuntimeWarning: Mean of empty slice
  video_score["cat_scores"][cat_id]["dice"] = np.nanmean(all_dice[cat_id]) if len(all_dice[cat_id]) > 0 else np.nan


In [11]:
result = {"videos": video_scores, "cat_scores": {cat_id: {"iou": None, "mae": None, "dice": None} for cat_id in cocoDT.getCatIds()},
        "avg_scores": {"iou": None, "mae": None, "dice": None},}

In [12]:
all_iou = {cat_id: [] for cat_id in cocoDT.getCatIds()}
all_mae = {cat_id: [] for cat_id in cocoDT.getCatIds()}
all_dice = {cat_id: [] for cat_id in cocoDT.getCatIds()}

for video_data in result["videos"]:
    for category_id, category_data in video_data["cat_scores"].items():
        all_iou[category_id].append(category_data["iou"]) 
        all_mae[category_id].append(category_data["mae"])
        all_dice[category_id].append(category_data["dice"])
        
for key in cocoDT.getCatIds():
    result["cat_scores"][key]['iou'] = np.nanmean(all_iou[key]) 
    result["cat_scores"][key]['mae'] = np.nanmean(all_mae[key])
    result["cat_scores"][key]['dice'] = np.nanmean(all_dice[key])
    
# print(all_iou)
    
avg_iou = []
avg_mae = []
avg_dice = []

for cat_id in cocoDT.getCatIds():
    result["cat_scores"][cat_id]["iou"] = np.nanmean(all_iou[cat_id]) if len(all_iou[cat_id]) > 0 else np.nan
    result["cat_scores"][cat_id]["mae"] = np.nanmean(all_mae[cat_id]) if len(all_mae[cat_id]) > 0 else np.nan
    result["cat_scores"][cat_id]["dice"] = np.nanmean(all_dice[cat_id]) if len(all_dice[cat_id]) > 0 else np.nan
    
    avg_iou.append(result["cat_scores"][cat_id]["iou"])
    avg_mae.append(result["cat_scores"][cat_id]["mae"])
    avg_dice.append(result["cat_scores"][cat_id]["dice"])


result["avg_scores"]["iou"] = np.nanmean(avg_iou) if len(avg_iou) > 0 else np.nan
result["avg_scores"]["mae"] = np.nanmean(avg_mae) if len(avg_mae) > 0 else np.nan
result["avg_scores"]["dice"] = np.nanmean(avg_dice) if len(avg_dice) > 0 else np.nan

In [13]:
result['avg_scores'] 

{'iou': np.float64(0.2978882613381307),
 'mae': np.float64(2.0186018619548944),
 'dice': np.float64(0.3735391090851365)}

In [15]:
from eval import eval
eval("output/points/predict.json", "coco_annotations.json", "output")

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
